# Use RNN to model COVID-19 Culmulative Cases

## Prepare training data set

In [1]:
import os
os.chdir("../")
import sys
sys.path.insert(0, ".")

from utils import *
from utils.paths import *
import utils.paths as p
from utils.plot import *

In [3]:
create_paths("DateWorld")
countries, index_country, country_index = get_countries()

In [19]:
df = pd.read_csv(f"{p.DRP}/{index_country[173]}.csv")
df = df.drop(['country', 'date', 'tagged_day'], axis=1)
df.iloc[0]['confirmed']

51

In [20]:
def time_series(df, window):
    X, y = [], []
    for i in range(0, len(df)-window):
        X.append(df.iloc[i:i+window].values)
        y.append(df.iloc[i+window]['confirmed'])
    return np.array(X), np.array(y)

In [21]:
X, y = time_series(df, 10)

In [77]:
factor = 1500000
X = X/factor
y = y/factor

In [79]:
sep1 = (int)(len(X) * 0.6)
sep2 = (int)(len(X) * 0.8)
X_train, y_train = X[:sep1], y[:sep1]
X_test, y_test = X[sep1:sep2], y[sep1:sep2]
X_pred, y_pred = X[sep2:], y[sep2:]

In [ ]:
def my_data_loader(X, y, batch_size):
    

# Torch RNN

In [32]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [90]:
class CovidRnn(nn.Module):
    def __init__(self, feature_dim, hidden_dim, num_layers, output_size):
        super(CovidRnn, self).__init__()
        self.feature_dim = feature_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_size = output_size
             
        self.rnn = nn.RNN(feature_dim, hidden_dim, num_layers)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.activation = nn.Tanh()
    
    def forward(self, x, hidden):

        rnn_out, hidden = self.rnn(x, hidden)
        rnn_out = rnn_out.contiguous().view(-1,self.hidden_dim)
        out = self.fc(rnn_out)
        out = self.activation(out)
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        h0 = torch.randn(self.num_layers, batch_size, self.hidden_dim)
        return h0

In [91]:
??nn.RNN

In [92]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [93]:
feature_dim, hidden_dim, num_lstm_layers, output_size = 4, 4, 1, 1
model = CovidRnn(feature_dim, hidden_dim, num_lstm_layers, output_size)
model.to(device)

lr = 0.00000005

In [94]:
criterion = nn.MSELoss()

In [95]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [96]:
batch_size = 7
train_data_torch = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_data_torch = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
pred_data_torch = TensorDataset(torch.from_numpy(X_pred), torch.from_numpy(y_pred))

train_loader = DataLoader(train_data_torch, shuffle=False, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_data_torch, shuffle=False, batch_size=batch_size, drop_last=True)
pred_loader = DataLoader(pred_data_torch, shuffle=False, batch_size=batch_size, drop_last=True)

In [105]:
for ix, iy in train_loader:
    print(ix[0].shape)
    

torch.Size([10, 4])
torch.Size([10, 4])
torch.Size([10, 4])
torch.Size([10, 4])
torch.Size([10, 4])


In [109]:
epochs = 100
counter = 0
clip = 5
print_every = 1

min_loss = np.Inf
min_diff = np.Inf
seq_len = 10

saving_index = 0

model.train()
h = 0

for i in range(epochs):
    h = model.init_hidden(batch_size)
    #display(h.shape)
    for X, y in train_loader:
        counter += 1
        X = X.reshape(seq_len,batch_size,feature_dim).float()
        #display(X.shape)
        # h = tuple([e.data for e in h])
        X, y = X.to(device), y.to(device)
        model.zero_grad()
        output, h = model(X, h)
        # display(output)
        loss = criterion(output.squeeze(), y.float())
        loss.backward(retain_graph=True)
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            test_h = model.init_hidden(batch_size)
            test_losses = []
            model.eval()
            for test_X, test_y in test_loader:
                # test_h = tuple([e.data for e in test_h])
                test_X = test_X.reshape(seq_len,batch_size,feature_dim).float()
                test_X, test_y = test_X.to(device), test_y.to(device)
                test_out, test_h = model(test_X, test_h)
                test_loss = criterion(test_out.squeeze(), y.float())
                test_losses.append(test_loss.item())
                
            model.train()
            print("Epoch: {}/{}".format(i+1, epochs),
                  "Step: {}".format(counter),
                  "Loss: {}".format(loss.item()),
                  "Val Loss: {}".format(np.mean(test_losses)))
            
            if abs(np.mean(test_losses)) <= min_loss or abs(np.mean(test_losses)-loss.item()) < min_diff:
                print(f"Saving {saving_index}")
                torch.save(model.state_dict(), f'./state_dict{saving_index}.pt')
                min_loss = np.mean(test_losses)
                min_diff = np.mean(test_losses)-loss.item()
                saving_index += 1
            

Epoch: 1/100 Step: 1 Loss: 0.004009926225990057 Val Loss: 0.00853828527033329
Saving 0
Epoch: 1/100 Step: 2 Loss: 0.0018768971785902977 Val Loss: 0.003631303319707513
Saving 1
Epoch: 1/100 Step: 3 Loss: 0.0043858750723302364 Val Loss: 0.0070287613198161125
Epoch: 1/100 Step: 4 Loss: 0.020195428282022476 Val Loss: 0.028872227296233177
Epoch: 1/100 Step: 5 Loss: 0.07649634778499603 Val Loss: 0.07356136292219162
Epoch: 2/100 Step: 6 Loss: 0.0015409215120598674 Val Loss: 0.007288489956408739
Epoch: 2/100 Step: 7 Loss: 0.0018757299985736609 Val Loss: 0.00403704633936286
Epoch: 2/100 Step: 8 Loss: 0.0043856599368155 Val Loss: 0.012537849135696888
Epoch: 2/100 Step: 9 Loss: 0.020194973796606064 Val Loss: 0.033386413007974625
Epoch: 2/100 Step: 10 Loss: 0.0764954537153244 Val Loss: 0.07320892065763474
Epoch: 3/100 Step: 11 Loss: 0.0019283536821603775 Val Loss: 0.006698396522551775
Epoch: 3/100 Step: 12 Loss: 0.0018747898284345865 Val Loss: 0.013671246357262135
Epoch: 3/100 Step: 13 Loss: 0.004

Epoch: 21/100 Step: 105 Loss: 0.07647888362407684 Val Loss: 0.08082183450460434
Epoch: 22/100 Step: 106 Loss: 0.0013684455770999193 Val Loss: 0.004183880984783173
Saving 23
Epoch: 22/100 Step: 107 Loss: 0.0018747021676972508 Val Loss: 0.004351456183940172
Saving 24
Epoch: 22/100 Step: 108 Loss: 0.004381540697067976 Val Loss: 0.010740499012172222
Epoch: 22/100 Step: 109 Loss: 0.020186159759759903 Val Loss: 0.022541966289281845
Saving 25
Epoch: 22/100 Step: 110 Loss: 0.0764780044555664 Val Loss: 0.08546353876590729
Epoch: 23/100 Step: 111 Loss: 0.0029976002406328917 Val Loss: 0.003993254154920578
Saving 26
Epoch: 23/100 Step: 112 Loss: 0.001874305191449821 Val Loss: 0.0038055831100791693
Saving 27
Epoch: 23/100 Step: 113 Loss: 0.004381336271762848 Val Loss: 0.025718869641423225
Epoch: 23/100 Step: 114 Loss: 0.020185718312859535 Val Loss: 0.02647939696907997
Epoch: 23/100 Step: 115 Loss: 0.07647713273763657 Val Loss: 0.07726182043552399
Saving 28
Epoch: 24/100 Step: 116 Loss: 0.0162557400

Epoch: 43/100 Step: 215 Loss: 0.07645978033542633 Val Loss: 0.08494091033935547
Epoch: 44/100 Step: 216 Loss: 0.0021678495686501265 Val Loss: 0.010446249507367611
Epoch: 44/100 Step: 217 Loss: 0.0018702175002545118 Val Loss: 0.0039368742145597935
Epoch: 44/100 Step: 218 Loss: 0.004377031698822975 Val Loss: 0.014412691816687584
Epoch: 44/100 Step: 219 Loss: 0.020176509395241737 Val Loss: 0.02665281668305397
Epoch: 44/100 Step: 220 Loss: 0.07645893096923828 Val Loss: 0.07525273412466049
Epoch: 45/100 Step: 221 Loss: 0.0022148783318698406 Val Loss: 0.01229052897542715
Epoch: 45/100 Step: 222 Loss: 0.00187165557872504 Val Loss: 0.0033679683692753315
Epoch: 45/100 Step: 223 Loss: 0.0043768309988081455 Val Loss: 0.007272628135979176
Epoch: 45/100 Step: 224 Loss: 0.020176073536276817 Val Loss: 0.02323930524289608
Epoch: 45/100 Step: 225 Loss: 0.07645805925130844 Val Loss: 0.08022834360599518
Epoch: 46/100 Step: 226 Loss: 0.0022921375930309296 Val Loss: 0.0038184409495443106
Epoch: 46/100 Step

Epoch: 63/100 Step: 315 Loss: 0.07644250243902206 Val Loss: 0.08734195679426193
Epoch: 64/100 Step: 316 Loss: 0.0018348239827901125 Val Loss: 0.015631604939699173
Epoch: 64/100 Step: 317 Loss: 0.0018674259772524238 Val Loss: 0.003654616652056575
Epoch: 64/100 Step: 318 Loss: 0.004372959025204182 Val Loss: 0.007085825782269239
Epoch: 64/100 Step: 319 Loss: 0.02016778104007244 Val Loss: 0.028338314965367317
Epoch: 64/100 Step: 320 Loss: 0.07644166052341461 Val Loss: 0.08020015060901642
Epoch: 65/100 Step: 321 Loss: 0.0027980634476989508 Val Loss: 0.00704615144059062
Epoch: 65/100 Step: 322 Loss: 0.001865689642727375 Val Loss: 0.0057136123068630695
Epoch: 65/100 Step: 323 Loss: 0.0043727559968829155 Val Loss: 0.008602256886661053
Epoch: 65/100 Step: 324 Loss: 0.02016735076904297 Val Loss: 0.0268305204808712
Epoch: 65/100 Step: 325 Loss: 0.07644081860780716 Val Loss: 0.07410929352045059
Epoch: 66/100 Step: 326 Loss: 0.0019609923474490643 Val Loss: 0.006104657892137766
Epoch: 66/100 Step: 3

Epoch: 85/100 Step: 424 Loss: 0.020158732309937477 Val Loss: 0.02197016216814518
Epoch: 85/100 Step: 425 Loss: 0.0764237493276596 Val Loss: 0.07621191442012787
Epoch: 86/100 Step: 426 Loss: 0.004815057851374149 Val Loss: 0.0047562154941260815
Epoch: 86/100 Step: 427 Loss: 0.0018632555147632957 Val Loss: 0.008323258720338345
Epoch: 86/100 Step: 428 Loss: 0.004368531052023172 Val Loss: 0.007429937366396189
Epoch: 86/100 Step: 429 Loss: 0.02015829272568226 Val Loss: 0.02121819369494915
Epoch: 86/100 Step: 430 Loss: 0.07642287760972977 Val Loss: 0.08358119428157806
Epoch: 87/100 Step: 431 Loss: 0.0060927048325538635 Val Loss: 0.005432221107184887
Epoch: 87/100 Step: 432 Loss: 0.001864703604951501 Val Loss: 0.005137948784977198
Epoch: 87/100 Step: 433 Loss: 0.004368327558040619 Val Loss: 0.007311173714697361
Epoch: 87/100 Step: 434 Loss: 0.02015784941613674 Val Loss: 0.024067293852567673
Epoch: 87/100 Step: 435 Loss: 0.07642201334238052 Val Loss: 0.08280978351831436
Epoch: 88/100 Step: 436 

In [110]:
model.load_state_dict(torch.load("state_dict40.pt"))

<All keys matched successfully>

In [132]:
i = []
pred = []
real = []
index = 0
for X, y in pred_loader:
    i.append(index)
    index += 1
    X = X.reshape(seq_len,batch_size,feature_dim).float()
    #display(X)
    # h_tmp = tuple([e.data for e in h])
    h_tmp = h
    X, y = X.to(device), y.to(device)
    model.zero_grad()
    output, _ = model(X, h_tmp)
    loss = criterion(output.squeeze(), y.float())
    display([o.item() for o in output])
    pred += [o.item() for o in output]
    real += [o.item() for o in y]
#     pred.append(factor*output.item().float())
#     real.append(factor*y.item())
#     print(f"prediction= {factor*output.item()}, real={factor*y.item()}, loss={loss}")

[-0.07783957570791245,
 -0.0761462152004242,
 -0.08255273848772049,
 -0.08501390367746353,
 -0.08651039749383926,
 -0.08771108835935593,
 -0.08932702243328094]

In [133]:
pred

[-0.07783957570791245,
 -0.0761462152004242,
 -0.08255273848772049,
 -0.08501390367746353,
 -0.08651039749383926,
 -0.08771108835935593,
 -0.08932702243328094]

In [134]:
real

[0.6438566666666666,
 0.658798,
 0.6750546666666667,
 0.6932726666666666,
 0.7129493333333333,
 0.7356406666666667,
 0.755026]